In [2]:
import pandas as pd
import numpy as np
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, PULP_CBC_CMD
from meteostat import Daily, Point
from datetime import datetime

# 📌 **Caricamento dati ritardi**
file_ritardi = "C:/Users/C.Marino/Desktop/dataset/ritardi_consistenti.txt"
df = pd.read_csv(file_ritardi, sep=",")

# 📌 **Conversione della colonna delay in minuti**
df['delay'] = pd.to_timedelta(df['delay'], errors='coerce').dt.total_seconds() / 60
df = df.dropna(subset=['delay'])  

# 📌 **Gestione della colonna 'hour'**
df['arrival_time_x'] = pd.to_datetime(df['arrival_time_x'], errors='coerce')
df['hour'] = df['arrival_time_x'].dt.hour  

# 📌 **Recupero dati meteo per Roma (10-14 febbraio 2025)**
start = datetime(2025, 2, 10)
end = datetime(2025, 2, 14)
rome = Point(41.9028, 12.4964)
weather = Daily(rome, start, end)
weather = weather.fetch()

# 📌 **Unione dei dati meteo con i dati ritardi**
weather = weather.reset_index()
weather['time'] = pd.to_datetime(weather['time'])
weather = weather[['time', 'tavg', 'prcp', 'wspd']]  # tavg = temperatura media, prcp = precipitazioni, wspd = velocità del vento
weather.rename(columns={'time': 'arrival_date'}, inplace=True)

df['arrival_date'] = pd.to_datetime(df['arrival_date'], errors='coerce')
df = df.merge(weather, on='arrival_date', how='left')

# 📌 **Gestione dei valori mancanti nei dati meteo**
df['tavg'].fillna(df['tavg'].mean(), inplace=True)
df['prcp'].fillna(0, inplace=True)  # Se pioggia non è riportata, assumiamo 0
df['wspd'].fillna(df['wspd'].mean(), inplace=True)

# 📌 **Raggruppamento per linea e ora per trovare le linee con più ritardi**
ritardi_per_linea = df.groupby(['route_id', 'hour'])[['delay', 'tavg', 'prcp', 'wspd']].mean().reset_index()
ritardi_per_linea = ritardi_per_linea.sort_values(by='delay', ascending=False)

# 🔍 **DEBUG: Controllo primi 20 ritardi per linea e ora**
print("\nPrimi 20 ritardi per linea e ora:")
print(ritardi_per_linea.head(20))

# 📌 **Parametri del modello prescrittivo**
max_extra_trips = 30  
max_per_line = 7  
min_trips_threshold = 180  

# 📌 **Seleziona le top 20 linee critiche**
critical_routes = ritardi_per_linea.head(20)

# 📌 **Creazione del problema di ottimizzazione**
model = LpProblem(name="Ottimizzazione-frequenze", sense=LpMaximize)

# 📌 **Definizione delle variabili decisionali**
trip_increase = { 
    (row['route_id'], row['hour']): LpVariable(name=f"extra_trips_{row['route_id']}_{int(row['hour'])}", lowBound=0, upBound=max_per_line, cat='Integer') 
    for _, row in critical_routes.iterrows()
}

# 🔍 **DEBUG: Controllo coefficienti della funzione obiettivo**
print("\nCoefficienti della funzione obiettivo:")
for key, var in trip_increase.items():
    delay_value = ritardi_per_linea.loc[
        (ritardi_per_linea['route_id'] == key[0]) & (ritardi_per_linea['hour'] == key[1]), 'delay'
    ].values
    if len(delay_value) > 0:
        print(f"Linea {key[0]}, Ora {key[1]} -> Coeff: {delay_value[0]}")
    else:
        print(f"Linea {key[0]}, Ora {key[1]} -> ⚠️ ERRORE: Nessun valore trovato!")

# 📌 **Funzione obiettivo: Massimizzare la riduzione dei ritardi considerando anche il meteo**
model += lpSum(
    trip_increase[(row['route_id'], row['hour'])] * (row['delay'] - 0.5 * row['prcp'] - 0.2 * row['wspd'])
    for _, row in critical_routes.iterrows()
), "Massimizzazione Riduzione Ritardi"

# 📌 **Vincoli del modello**
# 1️⃣ **Limite massimo di corse disponibili**
model += lpSum(trip_increase[(row['route_id'], row['hour'])] for _, row in critical_routes.iterrows()) <= max_extra_trips, "Limite Corso Totale"

# 2️⃣ **Garantiamo almeno 1 corsa alle linee con ritardi sopra la soglia**
for _, row in critical_routes.iterrows():
    if row['delay'] >= min_trips_threshold:
        model += trip_increase[(row['route_id'], row['hour'])] >= 1, f"Minimo_corse_{row['route_id']}_{row['hour']}"

# 📌 **Risoluzione del problema di ottimizzazione**
model.solve(PULP_CBC_CMD(msg=True))

# 📌 **Valore finale della funzione obiettivo**
print(f"\n✅ Valore finale della funzione obiettivo: {model.objective.value()}")

# 📌 **Estrazione delle nuove frequenze suggerite**
optimized_frequencies = {key: var.varValue for key, var in trip_increase.items()}

# 📌 **Creazione DataFrame con i risultati**
optimized_df = pd.DataFrame.from_dict(optimized_frequencies, orient='index', columns=['extra_trips'])
optimized_df.index = pd.MultiIndex.from_tuples(optimized_df.index, names=['route_id', 'hour'])

# 🔍 **DEBUG: Visualizzazione delle nuove frequenze**
print("\n🚀 Nuove frequenze suggerite:")
print(optimized_df)

# 📌 **Salvataggio risultati**
output_file = f"C:/Users/C.Marino/Desktop/dataset/ottimizzazione_frequenze_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.txt"
optimized_df.to_csv(output_file, sep=",")
print(f"\n📂 Risultati salvati in: {output_file}")

# 📌 Join con i dati originali per avere anche ritardi e meteo
final_df = optimized_df.reset_index()
final_df = final_df.merge(critical_routes, on=['route_id', 'hour'], how='left')

# 📌 Calcolo dell’effetto stimato sulla riduzione del ritardo
final_df['estimated_delay_reduction'] = final_df['extra_trips'] * (final_df['delay'] - 0.5 * final_df['prcp'] - 0.2 * final_df['wspd'])

# 📌 Riordino e rinomina colonne
final_df = final_df[['route_id', 'hour', 'delay', 'prcp', 'wspd', 'extra_trips', 'estimated_delay_reduction']]
final_df.columns = ['route_id', 'hour', 'delay', 'prcp', 'wspd', 'extra_trips', 'estimated_impact']

# 📌 Salvataggio del file completo per la dashboard
output_file_dashboard = f"C:/Users/C.Marino/Desktop/dataset/ottimizzazione_dashboard_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.csv"
final_df.to_csv(output_file_dashboard, sep=",", index=False)

print(f"\n📁 File per la dashboard salvato in: {output_file_dashboard}")




Primi 20 ritardi per linea e ora:
    route_id  hour       delay       tavg       prcp       wspd
129      350    11  265.709524  11.200000  13.800000  15.600000
131      360    10  226.785185  10.333333   5.666667   9.133333
176      507     9  221.074074  10.411111   4.155556   9.266667
130      360     9  215.817949  10.269231   5.230769   8.907692
307      766    11  214.946667  11.000000   3.300000  10.900000
359      913     9  210.613580   9.796296   0.000000   6.200000
177      507    10  203.448718  10.246154   3.676923   8.276923
306      766    10  202.922444  10.270916   3.156972   8.138645
84       201    11  200.510692  10.418868   3.424528   8.949057
305      766     9  195.650494  10.381111   4.590370   9.048148
184      515    11  195.566667  11.200000  13.800000  15.600000
183      515    10  192.292754  11.047826   9.091304  13.352174
74     19NAV     8  187.888889   9.666667   0.000000   6.200000
302       75    11  186.457724  11.200000  13.800000  15.600000
304  